# 01 Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import os

# 02 Importing Data

In [2]:
path = r'/Users/woodoooo/Desktop/Instacart Basket Analysis/'
df_ords = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'orders_wrangled_task_4.4.csv'), index_col = False)

# 03 Descriptive Statistics & Exploratory analysis for orders_wrangled_task_4.4.csv

In [3]:
df_ords.describe()

,Unnamed: 0,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order
count,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.214874e+06
mean,1.710541e+06,1.710542e+06,1.029782e+05,1.715486e+01,2.776219e+00,1.345202e+01,1.111484e+01
std,9.875817e+05,9.875817e+05,5.953372e+04,1.773316e+01,2.046829e+00,4.226088e+00,9.206737e+00
min,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.552705e+05,8.552715e+05,5.139400e+04,5.000000e+00,1.000000e+00,1.000000e+01,4.000000e+00
50%,1.710541e+06,1.710542e+06,1.026890e+05,1.100000e+01,3.000000e+00,1.300000e+01,7.000000e+00
75%,2.565812e+06,2.565812e+06,1.543850e+05,2.300000e+01,5.000000e+00,1.600000e+01,1.500000e+01
max,3.421082e+06,3.421083e+06,2.062090e+05,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01


After running df_ords.describe(), I checked the min and max values of each column to identify any unusual data:

Unnamed: 0
➔ Likely unnecessary index column. Min of 0 and max matching row count confirms this should be dropped.

order_id and user_id
➔ Min and max values seem logical. No immediate issues.

order_number
➔ Min: 1 (first order), Max: 100
➔ 100 seems like a reasonable maximum number of orders for returning customers.

order_day_of_week
➔ Min: 0, Max: 6
➔ This represents the seven days of the week (0 = Sunday, 6 = Saturday). Values are correct.

order_hour_of_day
➔ Min: 0, Max: 23
➔ Represents 24 hours in a day. Values are correct.

days_since_prior_order
➔ Min: 0, Max: 30
➔ 0 means same-day reorder. Max of 30 could indicate capping at 30 days, which seems reasonable.

In [4]:
df_ords.shape

(3421083, 7)

In [5]:
# Checking duplicates 
df_ords.duplicated().sum()

np.int64(0)

In [6]:
# Checking missing values
df_ords.isnull().sum()

Unnamed: 0                     0
order_id                       0
user_id                        0
order_number                   0
order_day_of_week              0
order_hour_of_day              0
days_since_prior_order    206209
dtype: int64

In [7]:
df_ords[df_ords['days_since_prior_order'].isnull()]['order_number'].value_counts()

order_number
1    206209
Name: count, dtype: int64

Checking whether 0 in days_since_prior_order means:

First order (no prior order), or

A reorder placed the same day as the previous order,

In [8]:
(df_ords['days_since_prior_order'] == 0).sum()

np.int64(67755)

Checking if Zeros Occur Beyond First Orders
If 0 appears in orders beyond the first one, then it must represent same-day reorders. To verify:

If the order_number > 1, then 0 = same-day reorder.

If the order_number = 1, then 0 is misrepresenting first orders.

In [9]:
df_ords[df_ords['days_since_prior_order'] == 0]['order_number'].value_counts().sort_index()

order_number
2      2899
3      2915
4      2964
5      2634
6      2443
       ... 
96      153
97      120
98      144
99      134
100     137
Name: count, Length: 99, dtype: int64

The output shows that 0 values appear in orders starting from order_number = 2 and upwards, never in order_number = 1. This confirms:

0 in days_since_prior_order means a same-day reorder.

First orders are properly represented as NaN (missing values). That's why we should keep all the NaN values.

In [10]:
# Checking for mixed types 
for col in df_ords.columns.tolist():
  weird = (df_ords[[col]].map(type) != df_ords[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_ords[weird]) > 0:
    print (col)

In [11]:
df_ords.dtypes

Unnamed: 0                  int64
order_id                    int64
user_id                     int64
order_number                int64
order_day_of_week           int64
order_hour_of_day           int64
days_since_prior_order    float64
dtype: object

In [12]:
# Based on the exploratory analysis we should only drop one column 
df_ords_cleaned = df_ords.drop('Unnamed: 0', axis=1)

In [13]:
df_ords_cleaned.shape

(3421083, 6)

In [16]:
df_ords_cleaned['new_customer'] = df_ords_cleaned['days_since_prior_order'].isnull() == True
df_ords_cleaned

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,new_customer
0,2539329,1,1,2,8,NaN,True
1,2398795,1,2,3,7,15.0,False
2,473747,1,3,3,12,21.0,False
3,2254736,1,4,4,7,29.0,False
4,431534,1,5,4,15,28.0,False
...,...,...,...,...,...,...,...
3421078,2266710,206209,10,5,18,29.0,False
3421079,1854736,206209,11,4,10,30.0,False
3421080,626363,206209,12,1,12,18.0,False
3421081,2977660,206209,13,1,12,7.0,False


# 04 Exporting Data

In [17]:
#Export the cleaned dataframe for df_ords
df_ords_cleaned.to_csv(os.path.join(path, '02 Data','Prepared Data', 'orders_cleaned.csv'))